In [1]:
# Ensure src/ is in sys.path for imports
import sys
import os
sys.path.insert(0, os.path.abspath('../src'))


# Stock Market Search Engine Demo

This notebook demonstrates four different stock market search engines:
1. Generic Search Engine API
2. DuckDuckGo Search
3. Traversaal Search
4. Local Qdrant Vector Search

Each engine has its own strengths and use cases:
- Generic Search: Broad web search with customizable parameters
- DuckDuckGo: Privacy-focused search with real-time results
- Traversaal: Specialized stock market search with rich metadata
- Qdrant Local: Fast semantic search with local data

In [2]:
# Install required packages
!pip install -r ../requirements.txt

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 18.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 21.4 MB/s eta 0:00:00
  Attempting uninstall: websockets
    Found existing installation: websockets 15.0.1
    Uninstalling websockets-15.0.1:
      Successfully uninstalled websockets-15.0.1
  Attempting uninstall: tomlkit
    Found existing installation: tomlkit 0.13.2
    Uninstalling tomlkit-0.13.2:
      Successfully uninstalled tomlkit-0.13.2
  Attempting uninstall: Pillow
    Found existing installation: pillow 11.0.0
    Uninstalling pillow-11.0.0:
      Successfully uninstalled pillow-11.0.0
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
  Attempting uninstall: aiofiles
    Found existing installation: aiofiles 24.1.0
    Uninstalling aiofiles-2

In [3]:
import os
import sys
import importlib
import pandas as pd
from dotenv import load_dotenv
from IPython.display import display, Markdown
import json

# Load environment variables
load_dotenv()

# Import base module first
from search_engines import base
importlib.reload(base)

# Import and reload search engine modules
import search_engines.generic as generic
import search_engines.duckduckgo as duckduckgo
import search_engines.traversaal as traversaal
import search_engines.qdrant_local as qdrant_local
import llm

# Reload all modules in reverse dependency order
importlib.reload(llm)
importlib.reload(qdrant_local)
importlib.reload(traversaal)
importlib.reload(duckduckgo)
importlib.reload(generic)

# Import search engines after reload
from search_engines.generic import GenericSearchEngine
from search_engines.duckduckgo import DuckDuckGoSearchEngine
from search_engines.traversaal import TraversaalSearchEngine
from search_engines.qdrant_local import QdrantLocalSearchEngine
from llm import OpenRouterLLM

/Users/kiran.ramanna/anaconda3/envs/ais_ml1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Initialize Search Engines and LLM

Each search engine requires different initialization parameters:
- Generic Search: Path to stock market data CSV
- DuckDuckGo: No API key needed
- Traversaal: API key
- Qdrant Local: Path to stock market data CSV
- OpenRouter LLM: API key for text generation

In [8]:
# Initialize search engines
engines = {}

# Generic Search Engine
try:
    engines['generic'] = GenericSearchEngine(data_path="../data/2022_03_17_02_06_nasdaq.csv")
    print("✓ Generic Search Engine initialized")
except Exception as e:
    print(f"✗ Generic Search Engine failed: {str(e)}")

# DuckDuckGo Search Engine
try:
    engines['duckduckgo'] = DuckDuckGoSearchEngine()
    print("✓ DuckDuckGo Search Engine initialized")
except Exception as e:
    print(f"✗ DuckDuckGo Search Engine failed: {str(e)}")

# Traversaal Search Engine
try:
    engines['traversaal'] = TraversaalSearchEngine()
    print("✓ Traversaal Search Engine initialized")
except Exception as e:
    print(f"✗ Traversaal Search Engine failed: {str(e)}")

# Qdrant Local Search Engine
try:
    engines['qdrant'] = QdrantLocalSearchEngine(data_path="../data/2022_03_17_02_06_nasdaq.csv")
    print("✓ Qdrant Local Search Engine initialized")
except Exception as e:
    print(f"✗ Qdrant Local Search Engine failed: {str(e)}")

# Initialize OpenRouter LLM
try:
    llm_client = llm.OpenRouterLLM()
    print("✓ OpenRouter LLM initialized")
except Exception as e:
    print(f"✗ OpenRouter LLM failed: {str(e)}")


Loaded existing embeddings from storage
✓ Generic Search Engine initialized
✓ DuckDuckGo Search Engine initialized
✗ Traversaal Search Engine failed: TRAVERSAAL_API_KEY environment variable not set
Using existing Qdrant collection
✓ Qdrant Local Search Engine initialized
✗ OpenRouter LLM failed: OPENROUTER_API_KEY environment variable not set


## Search Function

This function will run a search query across all initialized engines, display the raw results, and then show an LLM-processed summary of the results.

In [9]:
import numpy as np
# Helper to convert numpy types to native Python types
def convert_np(obj):
    if isinstance(obj, dict):
        return {k: convert_np(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_np(i) for i in obj]
    elif isinstance(obj, np.generic):
        return obj.item()
    else:
        return obj

def search_stocks(query: str, top_k: int = 1):
    """Search for stocks using all available engines.
    
    Args:
        query: Search query string
        top_k: Number of results to return per engine
    """
    results = {}
    
    # Run search on each engine
    for name, engine in engines.items():
        try:
            results[name] = engine.search(query, top_k=top_k)
        except Exception as e:
            print(f"Error with {name} engine: {str(e)}")
    
    # Display results
    for name, stocks in results.items():
        display(Markdown(f"### {name.upper()} Results"))
        
        for i, stock in enumerate(stocks, 1):
            display(Markdown(f"#### {i}. {stock.title}"))
            display(Markdown(f"**Score:** {stock.score:.2f}"))
            display(Markdown(f"**URL:** {stock.url}"))
            display(Markdown(f"**Snippet:** {stock.snippet}"))
            
            if stock.metadata:
                display(Markdown("**Metadata:**"))
                for key, value in stock.metadata.items():
                    if value:  # Only show non-empty values
                        display(Markdown(f"- {key}: {value}"))
            
            # Display raw response if available
            if stock.raw_response:
                display(Markdown("**Raw Response:**"))
                from IPython.display import JSON
                display(JSON(stock.raw_response))
            
            # Get LLM analysis for this stock
            try:
                context = {
                    "query": query,
                    "engine": name,
                    "stock": {
                        "title": stock.title,
                        "score": stock.score,
                        "url": stock.url,
                        "snippet": stock.snippet,
                        "metadata": stock.metadata,
                        "raw_response": stock.raw_response
                    }
                }
                
                llm_response = llm_client.generate(
                    prompt="Analyze these stock search results and provide insights about:",
                    context=convert_np(context)
                )
                
                # Add a dotted line for clear demarcation before LLM analysis
                display(Markdown('<hr style="border-top: 1px dotted #bbb;">'))
                # Underline the LLM Analysis heading
                display(Markdown('<u>**LLM Analysis:**</u>'))
                display(Markdown(llm_response))
            except Exception as e:
                display(Markdown(f"*Error getting LLM analysis: {str(e)}*"))
            
            display(Markdown("---"))

## Example Searches

Let's try an example search to see how each engine performs and how the LLM analyzes the results.

In [10]:
# Search for high-growth technology stocks
# search_stocks("Apple stock")

In [11]:
import importlib
import gradio_app
importlib.reload(gradio_app)
from gradio_app import launch_gradio_app
launch_gradio_app(engines, llm_client)

NameError: name 'llm_client' is not defined